In [1]:
import cv2
import sqlite3
import sys
import yaml
import time
from pathlib import Path
import numpy as np
import struct

class MP4ToGlimBag:
    def __init__(self, mp4_path, output_path, topic_name='/image'):
        self.mp4_path = mp4_path
        self.output_path = Path(output_path)
        self.topic_name = topic_name
        self.output_path.mkdir(parents=True, exist_ok=True)
        
    def convert(self):
        # Video file open and processing code remains the same...
        cap = cv2.VideoCapture(self.mp4_path)
        if not cap.isOpened():
            raise RuntimeError(f"Could not open video file: {self.mp4_path}")
            
        fps = cap.get(cv2.CAP_PROP_FPS)
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        frame_interval_ns = int(1e9 / fps)
        
        db_path = self.output_path / "data.db3"
        conn = sqlite3.connect(str(db_path))
        self._create_db_tables(conn)
        
        start_time = time.time_ns()
        message_count = 0
        
        while True:
            ret, frame = cap.read()
            if not ret:
                break
                
            success, buffer = cv2.imencode('.jpg', frame)
            if not success:
                continue
                
            timestamp = start_time + int(message_count * frame_interval_ns)
            frame_data = self._serialize_compressed_image(buffer, timestamp)
            
            self._insert_message(conn, timestamp, frame_data, message_count)
            message_count += 1
            
            if message_count % 100 == 0:
                print(f"Processed {message_count} frames...")
        
        # Metadata generation code remains the same...
        duration_ns = frame_count * frame_interval_ns
        metadata = {
            'rosbag2_bagfile_information': {
                'compression_format': '',
                'compression_mode': '',
                'custom_data': {
                    'glim_info': {
                        'frame_info': {
                            'width': width,
                            'height': height,
                            'fps': fps
                        }
                    }
                },
                'duration': {
                    'nanoseconds': duration_ns
                },
                'files': [{
                    'duration': {
                        'nanoseconds': duration_ns
                    },
                    'message_count': message_count,
                    'path': str(db_path.name),
                    'starting_time': {
                        'nanoseconds_since_epoch': start_time
                    }
                }],
                'message_count': message_count,
                'relative_file_paths': [str(db_path.name)],
                'starting_time': {
                    'nanoseconds_since_epoch': start_time
                },
                'storage_identifier': 'sqlite3',
                'topics_with_message_count': [{
                    'message_count': message_count,
                    'topic_metadata': {
                        'name': self.topic_name,
                        'offered_qos_profiles': '',
                        'serialization_format': 'cdr',
                        'type': 'sensor_msgs/msg/CompressedImage'
                    }
                }],
                'version': 6
            }
        }
        
        with open(self.output_path / "metadata.yaml", 'w') as f:
            yaml.dump(metadata, f, default_flow_style=False)
            
        cap.release()
        conn.close()
        print(f"Conversion complete! Processed {message_count} frames.")

    def _create_db_tables(self, conn):
        c = conn.cursor()
        c.execute('''CREATE TABLE topics
                    (id INTEGER PRIMARY KEY,
                     name TEXT NOT NULL,
                     type TEXT NOT NULL,
                     serialization_format TEXT NOT NULL,
                     offered_qos_profiles TEXT NOT NULL)''')
        c.execute('''CREATE TABLE messages
                    (id INTEGER PRIMARY KEY,
                     topic_id INTEGER NOT NULL,
                     timestamp INTEGER NOT NULL,
                     data BLOB NOT NULL)''')
        c.execute('''INSERT INTO topics (id, name, type, serialization_format, offered_qos_profiles)
                    VALUES (?, ?, ?, ?, ?)''',
                    (1, self.topic_name, 'sensor_msgs/msg/CompressedImage', 'cdr', ''))
        conn.commit()

    def _add_padding(self, data, alignment):
        """Add padding bytes to ensure proper alignment"""
        pad_len = (alignment - (len(data) % alignment)) % alignment
        return data + b'\0' * pad_len

    def _serialize_compressed_image(self, jpeg_buffer, timestamp):
        # CDR header (little endian)
        data = bytearray([0x00, 0x01, 0x00, 0x00])
        
        # std_msgs/Header
        sec = timestamp // 1_000_000_000
        nsec = timestamp % 1_000_000_000
        
        # Header timestamp
        data.extend(struct.pack('<ii', int(sec), int(nsec)))
        
        # Header frame_id
        frame_id = "camera"
        data.extend(struct.pack('<I', len(frame_id)))
        data.extend(frame_id.encode())
        data = self._add_padding(data, 4)  # Align to 4 bytes
        
        # CompressedImage format string
        format_str = "jpeg"
        data.extend(struct.pack('<I', len(format_str)))
        data.extend(format_str.encode())
        data = self._add_padding(data, 4)  # Align to 4 bytes
        
        # CompressedImage data array
        jpeg_bytes = jpeg_buffer.tobytes()
        data.extend(struct.pack('<I', len(jpeg_bytes)))
        data.extend(jpeg_bytes)
        data = self._add_padding(data, 8)  # Final alignment
        
        return bytes(data)
    
    def _insert_message(self, conn, timestamp, data, message_id):
        c = conn.cursor()
        c.execute('''INSERT INTO messages (id, topic_id, timestamp, data)
                    VALUES (?, ?, ?, ?)''',
                    (message_id + 1, 1, timestamp, data))
        conn.commit()

def main():
    mp4_path = 'output_video.mp4'
    output_dir = 'output_video'
    converter = MP4ToGlimBag(mp4_path, output_dir)
    converter.convert()

if __name__ == '__main__':
    main()

Processed 100 frames...
Processed 200 frames...
Processed 300 frames...
Processed 400 frames...
Processed 500 frames...
Processed 600 frames...
Processed 700 frames...
Conversion complete! Processed 798 frames.
